# Gallery Example: Hyper/Hyper/1 Reentrant Queue

This example demonstrates a HyperExp/HyperExp/1 queueing system with reentrant routing:
- **Arrivals**: Hyperexponential distribution (very high SCV)
- **Service**: Hyperexponential distribution (high SCV)
- **Servers**: 2 servers
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

This model has high variability in both arrivals and service.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hyphyp1_reentrant():
    """Create Hyper/Hyper/1-Reentrant queueing model"""
    model = Network('Hyper/Hyper/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    queue.set_number_of_servers(2)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # HyperExp arrival (SCV=64) for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
    source.set_arrival(oclass2, Disabled())
    
    # HyperExp service (SCV=4) for Class1, Exponential for Class2
    queue.set_service(oclass1, HyperExp.fit_mean_and_scv(0.5, 4))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_hyphyp1_reentrant()

## About High Variability Systems

This model combines:
- **Very high arrival variability** (SCV = 64): Extremely bursty arrivals
- **High service variability** (SCV = 4): Highly variable service times
- **Multiple servers** (k = 2): Provides some capacity buffering

The combination of high variability in both arrival and service processes leads to:
- Very long tails in waiting time distribution
- Unpredictable queue behavior
- Periods of extreme congestion

The multiple servers help mitigate some of this variability.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)